# (1) 시청 기록 데이터 로드

In [48]:
import pandas as pd 

watched=pd.read_json(r"C:\Users\USER\Desktop\랩실 자료\[프로젝트] 킬링 타임 방지를 위한 유튜브 알고리즘 시각화 및 조정\코드 파일\시청 기록 파일\[지애]시청 기록.json", encoding='utf-8')

In [49]:
print(len(watched))

44700


# (2) 데이터 구성

- header (Youtube Music / Youtube)  # 쇼츠 구분은 방법을 찾아봐야할 듯
- title : 영상 제목
- titleUrl : 영상  URL
- subtitles (채널이름 / 채널 주소)
- time : 시청 시각
- products, activityControls는 youtube

In [50]:
watched.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44700 entries, 0 to 44699
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   header            44700 non-null  object
 1   title             44700 non-null  object
 2   titleUrl          44435 non-null  object
 3   subtitles         42594 non-null  object
 4   time              44700 non-null  object
 5   products          44700 non-null  object
 6   activityControls  44700 non-null  object
dtypes: object(7)
memory usage: 2.4+ MB


'description', 'details'이 있는 경우도 있고 없는 경우도 있어서 자동화할려면 try exeption 필요!!!!!!!

In [51]:
# watched=watched.drop(['description', 'details'], axis=1)

In [52]:
print(watched.isnull().sum())

header                 0
title                  0
titleUrl             265
subtitles           2106
time                   0
products               0
activityControls       0
dtype: int64


In [53]:
watched=watched.dropna()

In [54]:
print(watched.isnull().sum())

header              0
title               0
titleUrl            0
subtitles           0
time                0
products            0
activityControls    0
dtype: int64


-----

## <컬럼 설명>

### 1. header
-> YouTube Music과 youtube를 분리해서 적용 가능

In [55]:
print(list(watched['header'].unique()))

['YouTube', 'YouTube Music']


In [56]:
YouTube_video = watched[watched['header'] == 'YouTube']
YouTube_music=watched[watched['header'] == 'YouTube Music']

### 2. title

In [57]:
print(YouTube_video['title'][100:130])

164            𝐏𝐥𝐚𝐲𝐥𝐢𝐬𝐭 인스타에 한번쯤 들은 노래 모음 🪐 을(를) 시청했습니다.
165    10k a month with just a phone, wifi and 1-2 hr...
166    색소침착 피부 필수시청🚨 이거 제발... 한번만 써보세요... #shorts 을(를...
167                         다이어트중 저녁 약속 대처법 을(를) 시청했습니다.
168                            빙글빙글도는 다이아몬드 을(를) 시청했습니다.
169      올림머리 숑이만의 방법 #헤어 #hair #올림머리 #셀프헤어 을(를) 시청했습니다.
170                         우유를 자르면 이렇게 돼요🤍 을(를) 시청했습니다.
171    Help!! I’m obsessed with BADA LEE!!😭😍🙌🫶🔥 #bada...
172    [스우파2] #스모크챌린지 Choreo by 바다 l Smoke (Prod. Dyn...
173    [로켓펀치 연희] 치키치키~🎶 응??😲 끼릭끼릭?!! (Feat. 유키) 을(를) ...
174                   탕후루 먹을때 꼭 조심해야 하는 이유! 을(를) 시청했습니다.
175               얼굴안나오게 이쁜 하늘샷찍는 포즈 4가지 추천 을(를) 시청했습니다.
176    베네피트 신상✨샤인 글래스 컬러 립밤 4가지 전색상 발색 #shorts 을(를) 시...
177    #glassskin #skincare #skincareroutine #viral #...
178            'Better Things' 안무연습 에스파 카리나 을(를) 시청했습니다.
179            얼굴형 예뻐보이는 #앞머리 #사이드뱅 자르는 법❗️ 을(를) 시청했습니다.
180    가을감성 크래프트지 선물포장 | Gift wrapping ideas #shorts ...
181    텐트 피칭후 마시는 과일주...이거지....

* " 을(를) 시청했습니다."를 replace로 제거

-> 제목만 남김

In [58]:
# https로 시작하는 주소가 제목으로 나오는 경우는 삭제된 동영상 -> drop
YouTube_video = YouTube_video[~YouTube_video['title'].str.startswith("https")]
YouTube_video['title'] = YouTube_video['title'].str.replace("을(를) 시청했습니다.", "")
print(YouTube_video['title'][100:110])

164                        𝐏𝐥𝐚𝐲𝐥𝐢𝐬𝐭 인스타에 한번쯤 들은 노래 모음 🪐 
165    10k a month with just a phone, wifi and 1-2 hr...
166          색소침착 피부 필수시청🚨 이거 제발... 한번만 써보세요... #shorts 
167                                     다이어트중 저녁 약속 대처법 
168                                        빙글빙글도는 다이아몬드 
169                  올림머리 숑이만의 방법 #헤어 #hair #올림머리 #셀프헤어 
170                                     우유를 자르면 이렇게 돼요🤍 
171    Help!! I’m obsessed with BADA LEE!!😭😍🙌🫶🔥 #bada...
172    [스우파2] #스모크챌린지 Choreo by 바다 l Smoke (Prod. Dyn...
173            [로켓펀치 연희] 치키치키~🎶 응??😲 끼릭끼릭?!! (Feat. 유키) 
Name: title, dtype: object


### 3. titleUrl 

중요! titleUrl에서 영상 id를 추출할 수 있음

https://www.youtube.com/watch?v=d7sFBOXn5T0

-> v=d7sFBOXn5T0

=> 이걸 유튜브 api에 활용하면 영상 정보를 얻을 수 있음
![Alt text](image.png)

영상 제목, 채널 id, 영상 description, 좋아요, 싫어요 수, 댓글 수

* 여기서 description을 자연어 처리로 특정 주제로 연결시킬 수 있으면 좋을 것 같음

In [59]:
YouTube_video[['title','titleUrl']][100:110]

,title,titleUrl
164,𝐏𝐥𝐚𝐲𝐥𝐢𝐬𝐭 인스타에 한번쯤 들은 노래 모음 🪐,https://www.youtube.com/watch?v=wzONRaDYAdA
165,"10k a month with just a phone, wifi and 1-2 hr...",https://www.youtube.com/watch?v=gr9iEY3huu8
166,색소침착 피부 필수시청🚨 이거 제발... 한번만 써보세요... #shorts,https://www.youtube.com/watch?v=DwJsEkoZBYU
167,다이어트중 저녁 약속 대처법,https://www.youtube.com/watch?v=5MjOsZpIUJ4
168,빙글빙글도는 다이아몬드,https://www.youtube.com/watch?v=PLdtSKvWdLM
169,올림머리 숑이만의 방법 #헤어 #hair #올림머리 #셀프헤어,https://www.youtube.com/watch?v=j8nGC8ofjJg
170,우유를 자르면 이렇게 돼요🤍,https://www.youtube.com/watch?v=z-QMxynuyJI
171,Help!! I’m obsessed with BADA LEE!!😭😍🙌🫶🔥 #bada...,https://www.youtube.com/watch?v=ykpDa477fWo
172,[스우파2] #스모크챌린지 Choreo by 바다 l Smoke (Prod. Dyn...,https://www.youtube.com/watch?v=7Sakx1F8AIs
173,[로켓펀치 연희] 치키치키~🎶 응??😲 끼릭끼릭?!! (Feat. 유키),https://www.youtube.com/watch?v=lzajRu5sDE0


### 4. subtitles
- name : 채널명
- url : 채널 주소

-> 채널 정보까지 api로 따낼 수 있었던 것 같음

In [60]:
print(YouTube_video['subtitles'])

0        [{'name': '오늘의 립', 'url': 'https://www.youtube...
1        [{'name': '우렁각시친구', 'url': 'https://www.youtub...
2        [{'name': 'Abigail Rosenfeld', 'url': 'https:/...
3        [{'name': '제로크림', 'url': 'https://www.youtube....
4        [{'name': '소녀의행성 Girlsplanet', 'url': 'https:/...
                               ...                        
44694    [{'name': '차홍 CHAHONG', 'url': 'https://www.yo...
44695    [{'name': 'tvN D ENT', 'url': 'https://www.you...
44696    [{'name': '차홍 CHAHONG', 'url': 'https://www.yo...
44697    [{'name': 'DDIBOO띠부', 'url': 'https://www.yout...
44698    [{'name': 'DDIBOO띠부', 'url': 'https://www.yout...
Name: subtitles, Length: 34621, dtype: object


### 5. time

In [61]:
YouTube_video['time'][:5]

0    2023-09-25T06:38:08.835Z
1    2023-09-25T06:38:07.713Z
2    2023-09-25T06:38:05.680Z
3    2023-09-25T06:38:02.136Z
4    2023-09-25T06:37:59.933Z
Name: time, dtype: object